In [1]:
import re
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import datetime
from datetime import timedelta
import pandas as pd
import json
import dateutil.parser
from functions import (get_sector, changes_from_press, get_df, 
                       normalise_to_index, save_data, get_content_header)
final_date = datetime.date(2014, 1, 1)


C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = []
page_num = 1

ticker = 'pm'
sector = get_sector(ticker)

index_ticker = '^GSPC'

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

previous_items = None
past_last_date = False

while True:

    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    s = requests.Session()
    html = requests.get(f'https://www.pmi.com/investor-relations/press-releases-and-events/press-releases-overview?template=press-release&page={page_num}', headers=headers)

    soup = BeautifulSoup(html.content) 
    press_items = soup.find_all('div', attrs={'class':'ir-search-results--item'})
    

    
    for press_item in press_items:
        
        date = press_item.find('span', attrs={'class':'ir-search-results--date'}).text.lstrip().rstrip() 
        
        title = press_item.find('h3', attrs={'class':'ir-search-results--title'}).text.rstrip() 
        url = press_item.find('a').get('href')
        url = f'https://www.pmi.com{url}'
        
        date = dateutil.parser.parse(date).date()

        pct_change = changes_from_press(stock_hist, date, 1)
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        content = get_content_header(url, 'meta', {'name':'og:description'})
        data.append([str(date), title, url, content, pct_change, index_price, norm_price])

        if date < final_date:
            past_last_date = True
            break
    if past_last_date == True:
        break
    
    page_num = page_num + 1
    previous_items = press_items
    
save_data(data, ticker, sector)